In [8]:
import pandas as pd

df = pd.read_csv('C:/ML_project_predict_heart_disease/data/heart_2020_cleaned.csv')

In [9]:
# 중복행 확인 : 18078개
print(f'중복행 개수 : {df.duplicated().sum()}')

# 중복행 제거
df = df.drop_duplicates().reset_index(drop = True)

중복행 개수 : 18078


In [10]:
from sklearn.preprocessing import LabelEncoder
import re

### 라벨 인코딩(컬럼 내 유니크 값 두 개)
label_features = ['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke',
                'DiffWalking', 'Sex','PhysicalActivity',
                'Asthma', 'KidneyDisease', 'SkinCancer']

for feature in label_features:
    encoder = LabelEncoder()
    item = df[feature].unique()
    encoder.fit(item)
    df[feature] = encoder.transform(df[feature])

### 원핫 인코딩(컬럼 내 유니크 값 세 개 이상)
onehot_features = ['AgeCategory', 'Race', 'GenHealth', 'Diabetic']
df = pd.get_dummies(df, columns= onehot_features)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# # 이상치 제거 판단
# df = df[df['BMI'] < 70]
# df = df[df['SleepTime'] != 24]
# df.reset_index(drop = True, inplace = True)

df.to_csv('heart_2020_final.csv', index = False)